In [10]:
import dataset_handler as dh
import testing_utils as tu
from termcolor import colored
import re
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import pickle

In [2]:
priming_text_path = "data/priming_texts/gsm8k/codegen/func_eq_short.txt"
gsm8k_path = "data/grade-school-math/grade_school_math/data/train.jsonl"

current_dataset = dh.init_dataset_from_name(
        "gsm8k", primingtext_path=priming_text_path
    )

In [3]:
subset_data = current_dataset.data[:3000]

subset_data = [elem["question"] for elem in subset_data]

In [30]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

tu.set_all_seeds("codegen")

embedder = SentenceTransformer('all-mpnet-base-v2')

# Corpus with example sentences
corpus = subset_data
corpus_embeddings = embedder.encode(corpus)

# Perform kmean clustering
num_clusters = 6
#clustering_model = KMeans(n_clusters=num_clusters, random_state=42)
#clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster[0])
    print("")

Cluster  1
Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?

Cluster  2
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?

Cluster  3
A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?

Cluster  4
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

Cluster  5
Natalia s

In [17]:
def print_colored_pos(dist_list, pos):
    for i, dist in enumerate(dist_list):
        if i == pos:
            print(colored(dist, "cyan"))

In [18]:
def extract_eq(full_answer):
    eq_pattern = re.compile(r"<<([^>]+)>>")
    split_list = re.findall(eq_pattern, full_answer)

    eq_string = ""

    for i, eq in enumerate(split_list):
        eq_string += f" eq{i+1}: {eq}"

    return eq_string

In [19]:
import numpy as np
def print_cluster_closest(cluster_id):
    d = clustering_model.transform(corpus_embeddings)[:, cluster_id]
    ind = np.argsort(d)[::][:3]
    print(list(ind))
    for ex_i,i in enumerate(ind):
        #print_colored_pos(clustering_model.transform([corpus_embeddings[i]])[0], 1)
        print( f"def exercise{ex_i+1}():\n"
            + '    """'
            + current_dataset.data[i]["question"]
            + " Hint: use these equations"
            + extract_eq(current_dataset.data[i]["answer"])
            + '"""')
        print("\n")
        current_dataset.print_entry_from_idx(i)

In [8]:
#save to a pickle the clsutering model
import pickle
with open("clustering_model_6.pkl", "wb") as f:
    pickle.dump(clustering_model, f)

In [20]:
clustering_model = pickle.load(open("clustering_model_6.pkl", "rb"))

In [26]:
print_cluster_closest(5)

[1701, 2051, 1594]
def exercise1():
    """Sandra has a box of apples that weighs 120 pounds.  She's going to use half the weight in apples to make applesauce.  The rest will be used to make apple pies.  She needs 4 pounds of apples per pie.  How many pies will she be able to make? Hint: use these equations eq1: 120/2=60 eq2: 120-60=60 eq3: 60/4=15"""


Sandra has a box of apples that weighs 120 pounds.  She's going to use half the weight in apples to make applesauce.  The rest will be used to make apple pies.  She needs 4 pounds of apples per pie.  How many pies will she be able to make?
The box weighs 120 pounds and she's going to use half of it to make applesauce so she'll use 120/2 = <<120/2=60>>60 pounds for applesauce
The box weighs 120 pounds and she'll use 60 pounds for applesauce so that leaves 120-60 = <<120-60=60>>60 pounds of apples
She has 60 pounds of apples and she needs 4 pounds to make a pie so she can make 60/4 = <<60/4=15>>15 pies
#### 15
15

------------------------

In [27]:
from sklearn.decomposition import PCA

pca = PCA(2)

df = pca.fit_transform(corpus_embeddings)

In [28]:
import matplotlib.pyplot as plt
#kmeans = KMeans(n_clusters= 8, random_state=0)
 
#predict the labels of clusters.
#label = kmeans.fit_predict(df)
 
#Getting unique labels
u_labels = np.unique(cluster_assignment)
 
#plotting the results:
for i in u_labels:
    plt.scatter(df[cluster_assignment == i , 0] , df[cluster_assignment == i , 1] , label = i)
plt.legend()
plt.show()

NameError: name 'cluster_assignment' is not defined